<a href="https://colab.research.google.com/github/JonathanSum/Happy-Sugar-Life-Weekly-Training/blob/master/Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
clas FolderDataset(Dataset):
  """
  Creates a PyTorch dataset from folder, returning two tensor images.
  Args:
  main_dir : directory where images are stored.
  transform (optional) : torchvision transforms to be applied while making dataset
  """

# Here is a simple dataset class 
#that converts all our image in a folder to PyTorch dataset.

  def __init__(self, main_dir, transform=None):
      self.main_dir = main_dir
      self.transform = transform
      self.all_imgs = os.listdir(main_dir)
  def __len__(self):
      return len(self.all_imgs)
  
  def __getitem__(self, idx):
      img_loc = os.path.join(self.main_dir, self.all_imgs[idx]
      image = Image.open(imag_loc).convert("RGB")

      if self.transform is not None:
          tensor_image = self.transform(image)

      return tensor_image, tensor_image

In [ ]:
class ConvEncoder(nn.Module):
    """
    A simple COnvolution Encoder Model
    """

    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 16, (3, 3), padding=(1,1))
        self.relu1 = nn.ReLU(inplace=True)
        self.maxpool1 = nn.MaxPool2d((2,2))

        self.conv2 = nn.Conv2d(16, 32, (3, 3), padding = (1,1))
        self.relu2 = nn.ReLU(inplace=True)
        self.maxpool2 = nn.MaxPool2d((2,2))

        self.conv3 = nn.Conv2d(32, 64, (3, 3), padding = (1,1))
        self.relu3 = nn.ReLU(inplace=True)
        self.maxpool3 = nn.MaxPool2d((2,2))

        self.conv4 = nn.Conv2d(64, 128, (3, 3), padding = (1,1))
        self.relu4 = nn.ReLU(inplace=True)
        self.maxpool4 = nn.MaxPool2d((2,2))

        self.conv5 = nn.Conv2d(128, 256, (3, 3), padding = (1,1))
        self.relu5 = nn.ReLU(inplace=True)
        self.maxpool5 = nn.MaxPool2d((2,2))

        #The Model
    def forward(self, x):
        # Downscale the image with conv maxpool etc.
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool3(x)

        x = self.conv4(x)
        x = self.relu4(x)
        x = self.maxpool4(x)

        x = self.conv5(x)
        x = self.relu5(x)
        x = self.maxpool5(x)
        return x

In [ ]:
class ConvDecoder(nn.Module):
    """
    A simple Convolutional Decoder Model
    """

    def __init__(self):
        super().__init__()
        self.deconv1 = nn.ConvTranspose(256, 128, (2, 2), strider = (2,2))
        self.relu1 = nn.Relu(inplace = True)

        self.deconv2 = nn.ConvTransposed2d(128, 64, (2, 2), stride = (2,2))
        self.relu2 = nn.Relu(inplace = True)

        self.deconv3 = nn.ConvTransposed2d(64, 32, (2, 2), stride = (2,2))
        self.relu3 = nn.Relu(inplace = True)

        self.deconv4 = nn.ConvTransposed2d(32, 16, (2, 2), stride = (2,2))
        self.relu4 = nn.Relu(inplace = True)

        self.deconv5 = nn.ConvTransposed2d(16, 3, (2, 2), stride = (2,2))
        self.relu5 = nn.Relu(inplace = True)
    def forward(self, x):
        x = self.deconv1(x)
        x = self.relu1(x)

        x = self.deconv2(x)
        x = self.relu2(x)

        x = self.deconv3(x)
        x = self.relu3(x)

        x = self.deconv4(x)
        x = self.relu4(x)

        x = self.deconv5(x)
        x = self.relu5(x)
        return x


In [ ]:
import torch
import torchvision.transforms as T

transforms = T.Compose([T.ToTensor()])

full_dataset = FolderDataset("../data/", transforms)

train_size = 0.75
val_size = 1 - train_size

# Split data to train and test
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Create the train dataloader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 32, shuffle=True)

# Create the validation dataloader

# Create the full dataloader
full_loader = torch.utils.data.DataLoader(full_data, batch_size = 32)

loss_fn = nn.MSELoss()

encoder = ConvEncoder()
decoder = ConvDecoder()

device = "cuda"

# Shift models to GPU
encoder.to(device)
decoder.to(device)

# Both the encoder and decoder parameters
autoencoder_params = list(encoder.parameters())
optimizer = optim.Adam(autoencoder_params, lr=1e-3)   # Adam Optimizer

# Time to Train !!!
EPOCHS = 10
# Usual Training Loop
for epoch in tqdm(range(EPOCHS)):
        train_loss = train_step(encoder, decoder, train_loader, loss_fn, optimizer, device = device)
        print(f"Epochs = {epoch}, Training Loss : {train_loss}")
        val_loss =val_step(sencoder, decoder, val_loader, loss_fn, device=device)

        print(f"Epochs = {epoch}, Validation Loss : {val_loss}")

        #Simple Best Model saving
        if val_loss < max_loss:
            print("Validation Loss decreased, saving new best model")
            torch.save(encoder.state_dict(), "encoder_model.pt")
            torch.save(encoder.state_dict(), "decoder_model.pt")
# Save the feature representations.
EMBEDDING_SHAPE = (1, 256, 16, 16) # This we know from out encoder

# We need feature representations for complete dataset not just train and validation
# Hence we use full loader here.
embedding = create_embedding(encoder, full_loader, EMBEDDING_SHAPE, device)

# Convert embedding to numpy and save them
numpy_embedding = embeddding.cpu().detach().numpy()
numpy_images = numpy_embedding.shape[0]

# Save the embeddings for complete dataset, note just train
flattened_embedding = numpy_embedding.reshape((num_images, -1))
np.save("Data_embedding.npy", flattened_embedding)

def train_step(encoder, decoder, train_loader, loss_fn, optimizer, device):
    """
    Performs a single training step
    Args:
    encoder: A convolutional Encoder. E.g. torch_model ConvEncoder
    decoder: A convolutional Decoder. E.G. torch_model ConvDecoder
    train_loader: PyTorch dataloader, containing (images, images).
    loss_fn: PyTorch loss_fn, computes loss between 2 images. 
    optimizer: PyTorch optimizer.
    device: "cuda" or "cpu"
    Returns: Train Loss
    """
    # Set networks to train mode.
    encoder.train()
    decoder.train()

    for batch_idx, (train_img, target_img) in enumerate(train_loader):
        #Move images to device
        train_img = train_img.to(device)
        target_img = target_img.to(device)

        #Zero grad the optimizer
        optimizer.zero_grad()
        # Feed the train images to encoder
        enc_output = encoder(train_img)
        # The output of encoder is input to decoder !
        dec_output = decoder(enc_output)

        # Decoder output is reconstructed image
        # Compute loss with it and orginal image which is target image.
        loss = loss_fn(dec_output, target_img)
        # Backpropogate
        loss.backward()
        # Apply the optimizer to network by calling step.
        optimizer.step()
    # Return the loss
    return loss.item()

def val_step(encoder, decoder, val_loader, loss_fn, device):
  """
  Performs a single training step
  Args:
  encoder: A convolutional Encoder. E.g. torch_model ConvEncoder
  decoder: A convolutional Decoder. E.g. torch_model ConvDecoder
  val_loader: PyTorch dataloader, containing (images, images).
  loss_fn: PyTorch loss_fn, computes loss betwwen 2 images.
  device: "cuda" or "cpu"
  Return: Validation Loss
  """

  # Set to eval mode.
  encoder.eval()
  decoder.eval()

  # We don't need to compute gradients while validating.
  with torch.no_grad():
      for batch_idx, (train_img, target_img)
          # Move to device
          train_img = train_img.to(device)
          target_img = target_img.to(device)

          # Again as train. Feed encoder the train image.
          enc_output = encoder(train_img)
          # Decoder takes encoder output and reconstructs the image.
          dec_output = decoder(enc_output)

          # Validation loss for encoder and decoder
          loss = loss_fn(enc_output, dec_output)
  # Return the loss
  return loss.item()

def create_embedding(encoder, full_loader):
    """
    Creates embedding using encoder from dataloader.
    encoder: A convolutional Encoder. E.g. torch_model ConvEncoder
    full_loader: PyTorch dataloader, containing (images, images) over entire dataset.
    embedding_dim: Tuple (c, h, w) Dimension of embedding = output of encoder dimesntions.
    device: "cuda" or "cpu"
    Returns: embedding of size (num_images_in_loader + 1, c, h, w)
    """
    # Set encoder to eval mode.
    encoder.eval()
    #Just a place holder for our 0th image embedding.

    embedding = torch.randn(embedding_dim)

    # Again we do not compute loss here so. No gradients.
    with torch.no_grad():
        for batch_idx, (train_img, target_img) in enumerate(full_loader):
            # We can compute this on GPU. faster
            train_img = train_img.to(device)

            # Get encoder outputs and move outputs to cpu
            enc_output = encoder(train_img).cpu()
            # Keep adding these outputs to embeddings.
            embedding = torch.cat((embedding, enc_output), 0)

    # Return the embeddings
    return embedding

In [ ]:
def compute_similar_images(image, num_images, embedding, device):
    """
    Given an image and number of similar images to search.
    Returns the num_images closet neares images.
    Args: 
    image: Image whose similar images are to be found.
    num_images: Number of similar images to find.
    embedding : A (num_images, embedding_dim) Embedding of images learnt from auto-encoder.
    device : "cuda" or "cpu" device.
    """

    image_tensor = T.ToTensor()(image)
    image_tensor = image_tensor.unsqueeze(0)

    with torch.no_grad():
        image_embedding = encoder(image_tensor).cpu().detach().numpy()

    flattened_embedding = image_embedding.reshape((image_embedding.shape[0], -1))

    knn = NearestNeighbors(n_neighbors=num_images, metric="cosine")
    knn.fit(embedding)

    _, indices = knn.kneighbors(flattened_embedding)
    indices_list = indices.tolist()
    return indices_list

In [ ]:
#cite: https://t.co/fUUhCcPDhw?amp=1